In [1]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 


In [2]:
class GetData():
    def __init__(self):
        self.endpoint = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=1000"
        response = requests.get(url=self.endpoint)
        self.data = response.json()

In [4]:
test = GetData()

In [6]:
test.data[2]

{'transmission_datetime': '120744642_4_04042017090156',
 'post_id': '471-10460',
 'street_block': 'HARRISON ST 1000',
 'payment_type': 'CREDIT CARD',
 'session_start_dt': '2017-04-04T09:01:56.000',
 'session_end_dt': '2017-04-04T11:01:56.000',
 'meter_event_type': 'NS',
 'gross_paid_amt': '4.5'}

In [7]:
len(test_data.data)

1000